In [1]:
import os
#import dlib
import glob

import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import numpy as np
%matplotlib inline

In [2]:
img_black = np.zeros((70,140,3))

base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

classes = 'Begin, Choose, Connection, Navigation, Next, Previous, Start, Stop, Hello, Web'
classes = classes.split(', ')

predictor_path = 'assests/predictors/shape_predictor_68_face_landmarks.dat'

#detector = dlib.get_frontal_face_detector()
#predictor = dlib.shape_predictor(predictor_path)

train_people = 'F01, F02, F04, F05, F06, F07, F08, F09, M01, M02, M04'.split(', ')
val_people = 'F10, M07'.split(', ')
test_people = 'F11, M08'.split(', ')
print(train_people)
print(val_people)
print(test_people)

classes_num = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
word_ids = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
classes_dict = dict(zip(classes_num, classes))
print(classes_num)
print(word_ids)
print(classes_dict)

['F01', 'F02', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'M01', 'M02', 'M04']
['F10', 'M07']
['F11', 'M08']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
{'01': 'Begin', '02': 'Choose', '03': 'Connection', '04': 'Navigation', '05': 'Next', '06': 'Previous', '07': 'Start', '08': 'Stop', '09': 'Hello', '10': 'Web'}


In [3]:
def show_img(img, figsize=(8, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.grid(False)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.imshow(img)
    plt.imshow(img)

In [4]:
train_image = []
train_class = []

for class_id in tqdm(range(len(classes))):
    images = sorted(glob.glob('data/train/' + classes[class_id] + '/*.jpg'))
    for i in range(len(images)):
        if ('noised' in images[i] or 'rand_contr' in images[i]
        or 'vert_flip' in images[i] or 'hor_flip' in images[i]):
            train_image.append(images[i].split('/')[3])
            train_class.append(classes_dict[images[i].split('/')[3].split(']')[-1][:2]])
        else:
            train_image.append(images[i].split('/')[3])
            train_class.append(classes_dict[images[i].split('/')[3].split('_')[1]])
        
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

train_data.to_csv('data/miracl/train_new.csv',header=True, index=False)


val_image = []
val_class = []

for class_id in tqdm(range(len(classes))):
    images = sorted(glob.glob('data/validation/' + classes[class_id] + '/*.jpg'))
    for i in range(len(images)):
        val_image.append(images[i].split('/')[3])
        val_class.append(classes_dict[images[i].split('/')[3].split('_')[0]])
val_data = pd.DataFrame()
val_data['image'] = val_image
val_data['class'] = val_class

val_data.to_csv('data/miracl/val_new.csv',header=True, index=False)

100%|██████████| 10/10 [00:00<00:00, 679.60it/s]


In [5]:
train = pd.read_csv('data/miracl/train_new.csv')
val = pd.read_csv('data/miracl/val_new.csv')

y_tr = train['class']
y_tr_dummy = pd.get_dummies(y_tr)

y_val = val['class']
y_val_dummy = pd.get_dummies(y_val)

In [6]:
train.head()

,image,class
0,000000_[vert_flip]01_M04_02_color_001.jpg,Begin
1,000000_[vert_flip]01_M04_02_color_002.jpg,Begin
2,000000_[vert_flip]01_M04_02_color_003.jpg,Begin
3,000000_[vert_flip]01_M04_02_color_004.jpg,Begin
4,000000_[vert_flip]01_M04_02_color_005.jpg,Begin


In [7]:
train.tail()

,image,class
46867,000437_10_F05_08_color_008.jpg,Web
46868,000437_10_F05_08_color_009.jpg,Web
46869,000437_10_F05_08_color_010.jpg,Web
46870,000437_10_F05_08_color_011.jpg,Web
46871,000437_10_F05_08_color_012.jpg,Web


In [8]:
val.head()

,image,class
0,01_F10_01_color_001.jpg,Begin
1,01_F10_01_color_002.jpg,Begin
2,01_F10_01_color_003.jpg,Begin
3,01_F10_01_color_004.jpg,Begin
4,01_F10_01_color_005.jpg,Begin


In [9]:
val.tail()

,image,class
1505,10_M07_10_color_006.jpg,Web
1506,10_M07_10_color_007.jpg,Web
1507,10_M07_10_color_008.jpg,Web
1508,10_M07_10_color_009.jpg,Web
1509,10_M07_10_color_010.jpg,Web


In [10]:
print(len(train))
print(len(val))

46872
1510


In [11]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [12]:
import keras
#from tensorflow import keras

print(tf.__version__)
print(keras.__version__)

2.1.0
2.3.1


Using TensorFlow backend.


In [13]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [14]:
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.3
# set_session(tf.Session(config=config))

In [15]:
from tweaked_ImageGenerator_v2 import ImageDataGenerator

In [16]:
datagen = ImageDataGenerator()
train_data = datagen.flow_from_directory('data/train', target_size=(70, 140), batch_size=3, frames_per_step=12, shuffle=False)

Found 46872 images belonging to 10 classes.


In [17]:
val_data = datagen.flow_from_directory('data/validation', target_size=(70, 140), batch_size=3, frames_per_step=12, shuffle=False)

Found 1510 images belonging to 10 classes.


In [18]:
import math

print(math.ceil(46872/(3*12)))
print(math.ceil((1510/(3*12))))

1302
42


In [19]:
model_small = keras.models.Sequential()

model_small.add(keras.layers.ZeroPadding3D(padding=(1, 2, 2), input_shape=(12, 70, 140, 3)))
model_small.add(keras.layers.Conv3D(32, (3, 10, 10), strides=(1, 2, 2), activation='relu'))
model_small.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model_small.add(keras.layers.Dropout(.3))

model_small.add(keras.layers.ZeroPadding3D(padding=(1, 2, 2)))
model_small.add(keras.layers.Conv3D(64, (3, 5, 5), strides=(1, 1, 1), activation='relu'))
model_small.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model_small.add(keras.layers.Dropout(.3))

model_small.add(keras.layers.ZeroPadding3D(padding=(1, 1, 1)))
model_small.add(keras.layers.Conv3D(96, (3, 3, 3), strides=(1, 2, 2), activation='relu'))
model_small.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model_small.add(keras.layers.Dropout(.3))

model_small.add(keras.layers.TimeDistributed(keras.layers.Flatten()))

model_small.add(keras.layers.Bidirectional(keras.layers.LSTM(36, activation='relu', return_sequences=False)))

model_small.add(keras.layers.Dense(10, activation='softmax'))

In [20]:
adam2 = keras.optimizers.Adam(lr=0.0001)
save_weights = keras.callbacks.ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model_small.compile(loss='categorical_crossentropy',optimizer=adam2,metrics=['accuracy'])
model_small.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding3d_1 (ZeroPaddin (None, 14, 74, 144, 3)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 12, 33, 68, 32)    28832     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 12, 16, 34, 32)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 16, 34, 32)    0         
_________________________________________________________________
zero_padding3d_2 (ZeroPaddin (None, 14, 20, 38, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 12, 16, 34, 64)    153664    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 12, 8, 17, 64)    

In [21]:
history = model_small.fit_generator(train_data, epochs=20, steps_per_epoch=1302, 
                              validation_data=val_data, validation_steps=42, 
                              callbacks=[save_weights], shuffle=True)

Epoch 1/20
1302/1302 [==============================] - 919s 706ms/step - loss: 11.1349 - accuracy: 0.4568 - val_loss: 0.1996 - val_accuracy: 0.0861
Epoch 2/20
 294/1302 [=====>........................] - ETA: 11:42 - loss: 5.4944 - accuracy: 0.4566

KeyboardInterrupt: 